# Sieve of Randall

## Introduction

![alt text](./img/crypto.jpg "Introduction")

## Abstract

Last time I spoke with Randall, he was quite interested in a particular type of machines that
initially were designed to solve a problem and while doing so they were catalyzing its own
construction. He was pretty convinced that, as long as enough energy was around, any solvable
problem could be solved in that way. I couldn't find flaws in his logic, and we pursued an
exploration.

In fact, he pointed out that the machine will create (or better said, discover) new problems
on it’s way and solve them with the sole purpose of collecting and storing energy. And again,
he cleverly made a guess that different components of the machine will establish a symbiotic
relation to parallelize the resolution of the problems and to reuse previous solutions when
necessary.

That’s how the concept of RIST was born, which was the most natural way to study the behavior
of different parts of the machine over time. Something overlooked back then was the possibility
that the machine became aware of itself, in the sense that a RIST will develop enough
computing power to formulate a question about its nature of existence.

Randall once joked about that. “Ha! What if the machine now knows that something exists?
Is that a bug?”. And yes, consciousness was initially treated as a bug and it was used as
a stopping criteria for whatever problem was being solved.

But as a good friend of mine always says, a bug, it’s a feature. So we opened a new branch where
we let consciousness develop freely in the machine whenever it was possible.

Among other things, we found that the development of consciousness was quite different on
each iteration, depending on the initial problem. But regardless of that genesis trigger,
we found a set of problems that appeared over and over on every conscious instance of
the machine that we created. We didn’t probe it, but we had a good intuition that the
discovery of those problems is a necessary condition for consciousness to exist.

Since we were skipping our arts class at that time, we couldn’t think of a more creative name
than “Necessary Problems” for them. It was later changed to “Natural Problems” because no one
worked on a proof for that yet. Not even the machine $\zeta$

And here it is Randy, look at what you did !

## Prime Sieve Automata (PSA)

### Handcrafting the logic

Let's consider a language with an alphabet of "1" and "0" where natural numbers are expressed in the
base-2 numeral system. For example, 2 it's represented as "10" and 11 as "1011". The automata
we'll build is going to read the strings starting at the right most bit (least significant bit)
of the string.

In general, the following function converts a natural number into the binary
representation we'll work with.

In [1]:
def get_binary_input(number):
    binary = "{0:0128b}".format(number)
    stack = [c for c in binary[binary.find('1'):][::-1]]

    return stack

for i in range(1, 12):
    print("[+] binary stack for n =", i, get_binary_input(i))

[+] binary stack for n = 1 ['1']
[+] binary stack for n = 2 ['0', '1']
[+] binary stack for n = 3 ['1', '1']
[+] binary stack for n = 4 ['0', '0', '1']
[+] binary stack for n = 5 ['1', '0', '1']
[+] binary stack for n = 6 ['0', '1', '1']
[+] binary stack for n = 7 ['1', '1', '1']
[+] binary stack for n = 8 ['0', '0', '0', '1']
[+] binary stack for n = 9 ['1', '0', '0', '1']
[+] binary stack for n = 10 ['0', '1', '0', '1']
[+] binary stack for n = 11 ['1', '1', '0', '1']


Now let's build a finite automata which is going to accepts binary strings of prime numbers until 11.
A straightforward way would be to create a single final state ($ ip_0 $) and connect the
sequence of ones and zeroes from the initial state ($ is_0 $) for each prime.

In [2]:
from zephod.finite import *

initial, final = State("is0"), State("ip0")

delta = FADelta()

# number 2
delta.add(initial, State("z0"), {"0"})
delta.add(State("z0"), final, {"1"})

# number 3
delta.add(initial, State("z1"), {"1"})
delta.add(State("z1"), final, {"1"})

# number 5
delta.add(initial, State("z2"), {"1"})
delta.add(State("z2"), State("z3"), {"0"})
delta.add(State("z3"), final, {"1"})

# number 7
delta.add(initial, State("z4"), {"1"})
delta.add(State("z4"), State("z5"), {"1"})
delta.add(State("z5"), final, {"1"})

# number 11
delta.add(initial, State("z6"), {"1"})
delta.add(State("z6"), State("z7"), {"1"})
delta.add(State("z7"), State("z8"), {"0"})
delta.add(State("z8"), final, {"1"})

eleven_sieve = FiniteAutomata(transition=delta, initial=initial, final={final})

print("[+] the eleven prime sieve")
print(eleven_sieve)

from utils.plotter import *
from IPython.display import IFrame

print("\n[+] automata plotter")

AutomataPlotter.tikz(eleven_sieve, filename="eleven_sieve", output="./img/")
IFrame("./img/eleven_sieve.pdf", width=900, height=400)

[+] the eleven prime sieve
states = {z3, z6, z1, z5, z7, z2, z0, z8, z4, ip0, is0} ; alphabet = {'0', '1'}
is0 (0) -> {z0}
is0 (1) -> {z4, z2, z6, z1}
z0 (1) -> {ip0}
z1 (1) -> {ip0}
z2 (0) -> {z3}
z3 (1) -> {ip0}
z4 (1) -> {z5}
z5 (1) -> {ip0}
z6 (1) -> {z7}
z7 (0) -> {z8}
z8 (1) -> {ip0}
initial = is0 ; final = {ip0}

[+] automata plotter


The resulting automata is non deterministic (NFDA), since on the first state there are several
transitions with the same symbol ("1") going to several others.

We can use a prime sieve to verify that the automata works as expected.

In [3]:
from sympy import sieve

for i in range(2, 12):
    accepted = eleven_sieve.read(get_binary_input(i))
    if i in sieve:
        print("[+] checking prime number n =", i, "accepted =", accepted)
        assert accepted
    else:
        print("[+] checking composite number n =", i, "accepted =", accepted)
        assert not accepted

[+] checking prime number n = 2 accepted = True
[+] checking prime number n = 3 accepted = True
[+] checking composite number n = 4 accepted = False
[+] checking prime number n = 5 accepted = True
[+] checking composite number n = 6 accepted = False
[+] checking prime number n = 7 accepted = True
[+] checking composite number n = 8 accepted = False
[+] checking composite number n = 9 accepted = False
[+] checking composite number n = 10 accepted = False
[+] checking prime number n = 11 accepted = True


Given the construction method, what this automata is pretty much doing is a linear search into the
prime numbers that are encoded on its transition function. It will try each one of the 4 branches
that are coming out from $is_0$ until it gets to the final state, or it doesn't and rejects
the string for non prime numbers.

You can also debug the evolution of the automata for a given string, along with the different paths
is taking when is non deterministic.

In [4]:
eleven_sieve.debug(get_binary_input(11))

-------------------------------------------------------------------------------

initial (is0)         final {ip0}                                                                                       

[C0] 1 1 0 1               is0                           

-------------------------------------------------------------------------------
[C0] 1 1 0 1               [is0, 1(0)] to [z6, 1(1)]     
[C0] 1 1 0 1               [is0, 1(0)] to [z1, 1(1)]     
[C0] 1 1 0 1               [is0, 1(0)] to [z2, 1(1)]     
[C0] 1 1 0 1               [is0, 1(0)] to [z4, 1(1)]     
-------------------------------------------------------------------------------
[C0] 1 1 0 1               [z1, 1(1)] to [ip0, 0(2)]     
[C0] 1 1 0 1               [z4, 1(1)] to [z5, 0(2)]      
[C0] 1 1 0 1               [z6, 1(1)] to [z7, 0(2)]      
-------------------------------------------------------------------------------
[C0] 1 1 0 1               [z7, 0(2)] to [z8, 1(3)]      
----------------------------------

As a first step, we can convert this NFDA into a deterministic one, which is going to result in a
DFA where each state will have at most 2 transitions and the efficiency to determine if a number
is prime or not it will be much better than the NFDA case.

In [5]:
print("\n[+] deterministic FA eleven sieve")

deterministic_eleven_sieve = eleven_sieve.get_deterministic_automata()

AutomataPlotter.tikz(deterministic_eleven_sieve, filename="deterministic_eleven_sieve", output="./img/")
IFrame("./img/deterministic_eleven_sieve.pdf", width=900, height=300)


[+] deterministic FA eleven sieve


Much better and cleaner. We can further minimize the automata to reduce the number of states but in
this case it wouldn't make a difference since is already minimal.

### Moving it forward

What if we keep applying this process for larger numbers? In fact, we could keep reusing previous
minimized DFAs iteratively while adding new primes to its structure.

Let's do it for all natural numbers of 7 bits. We can easily automate the construction of the
automata with the following methods using our external sieve to find prime numbers.

In [6]:
class PrimeBuilder:
    def __init__(self, final_state):
        self.counter = 0
        self.final_state = final_state

    def get_new_state(self):
        self.counter += 1
        return State("z" + str(self.counter))

    def state_factorization(self, initial_state, transition, n):
        if n in sieve:
            stack, state = get_binary_input(n), initial_state
            for ic, b in enumerate(stack):
                current_state = self.get_new_state()

                if ic == 0:
                    transition.add(state, current_state, b)
                elif ic == len(stack) - 1:
                    transition.add(state, self.final_state, b)
                else:
                    transition.add(state, current_state, b)

                print(n, state, current_state, b)

                state = current_state

    def build(self, initial_state, transition, lower, higher):
        for lm in range(lower, higher):
            self.state_factorization(initial_state, transition, lm)

# get initial state from the previous automata
initial = eleven_sieve.initial
final = eleven_sieve.final.copy()
prime_state = State("ip0")

delta = copy.deepcopy(eleven_sieve.transition)

prime_builder = PrimeBuilder(final_state=prime_state)

# add number from 11 to 64 (7 bits)
prime_builder.build(initial, delta, 12, 65)

# build the new automata with the additional primes
nfda_seven_sieve = FiniteAutomata(transition=delta, initial=initial, final=final)

print("\n[+] seven bit NFDA  sieve")

AutomataPlotter.tikz(nfda_seven_sieve, filename="nfda_seven_sieve", output="./img/")
IFrame("./img/nfda_seven_sieve.pdf", width=900, height=1000)

[+] skipping repeated transition is0 -> z1 with C0 : move right (on = 1)
13 is0 z1 1
13 z1 z2 0
13 z2 z3 1
[+] skipping repeated transition z3 -> ip0 with C0 : move right (on = 1)
13 z3 z4 1
17 is0 z5 1
17 z5 z6 0
17 z6 z7 0
[+] skipping repeated transition z7 -> z8 with C0 : move right (on = 0)
17 z7 z8 0
[+] skipping repeated transition z8 -> ip0 with C0 : move right (on = 1)
17 z8 z9 1
19 is0 z10 1
19 z10 z11 1
19 z11 z12 0
19 z12 z13 0
19 z13 z14 1
23 is0 z15 1
23 z15 z16 1
23 z16 z17 1
23 z17 z18 0
23 z18 z19 1
29 is0 z20 1
29 z20 z21 0
29 z21 z22 1
29 z22 z23 1
29 z23 z24 1
31 is0 z25 1
31 z25 z26 1
31 z26 z27 1
31 z27 z28 1
31 z28 z29 1
37 is0 z30 1
37 z30 z31 0
37 z31 z32 1
37 z32 z33 0
37 z33 z34 0
37 z34 z35 1
41 is0 z36 1
41 z36 z37 0
41 z37 z38 0
41 z38 z39 1
41 z39 z40 0
41 z40 z41 1
43 is0 z42 1
43 z42 z43 1
43 z43 z44 0
43 z44 z45 1
43 z45 z46 0
43 z46 z47 1
47 is0 z48 1
47 z48 z49 1
47 z49 z50 1
47 z50 z51 1
47 z51 z52 0
47 z52 z53 1
53 is0 z54 1
53 z54 z55 0
53 z55 z56